### Imports

In [1]:
import duckdb
import os
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
from google.cloud import storage
import re

### Events

In [2]:
def duck_to_pandas(database_path, query):
    """
    Conecta ao DuckDB, executa a consulta SQL fornecida e retorna os resultados como um DataFrame do pandas.

    Args:
    - database_path (str): O caminho para o arquivo do banco de dados DuckDB.
    - query (str): A consulta SQL a ser executada.

    Returns:
    - pd.DataFrame: DataFrame contendo os resultados da consulta SQL.
    """
    try:
        # Conectar ao banco de dados DuckDB
        con = duckdb.connect(database=database_path, read_only=False)

        # Executar a consulta SQL e carregar os resultados em um DataFrame do pandas
        df = con.execute(query).df()

        # Fechar a conexão
        con.close()

        return df
    except Exception as e:
        print(f"Erro ao executar a consulta: {e}")
        return None

In [3]:
database_path = '/home/marcelo-borges/Documentos/Projetos/SIAN/laboratory/telemetry-validation/db/telemetria.duckdb'
query = "SELECT * FROM events"
df_events = duck_to_pandas(database_path, query)

### EDA

In [5]:
df_events.describe()

,TotalOccurances,TotalTimeSeconds,EventTypeId,EventId,DriverId,AssetId,Value,StartOdometerKilometres,FuelUsedLitres,EndOdometerKilometres,LocationId
count,1.238955e+06,1.238955e+06,1.238955e+06,1.238955e+06,1.238955e+06,1.238955e+06,1.205826e+06,9.181830e+05,381046.000000,6.378600e+05,1.206700e+04
mean,2.005445e+00,2.073852e+02,1.551935e+18,3.308199e+18,4.865914e+17,1.507137e+18,1.007198e+03,6.964791e+05,0.019021,6.924665e+05,1.508070e+18
std,1.317910e+01,1.397186e+03,5.584598e+18,1.285241e+16,1.217497e+18,1.386682e+16,4.812906e+03,5.094844e+05,0.045152,4.136210e+05,2.815040e+14
min,0.000000e+00,0.000000e+00,-8.078464e+18,3.275198e+18,-9.195140e+18,1.498924e+18,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,1.507979e+18
25%,1.000000e+00,1.000000e+00,-3.393531e+18,3.297818e+18,-1.066474e+17,1.498925e+18,0.000000e+00,5.489248e+05,0.003741,5.532453e+05,1.507980e+18
50%,1.000000e+00,3.000000e+00,1.443761e+18,3.308728e+18,-1.066474e+17,1.498925e+18,0.000000e+00,6.855360e+05,0.008728,8.499894e+05,1.507981e+18
75%,1.000000e+00,1.500000e+01,7.651958e+18,3.319500e+18,1.491699e+18,1.514417e+18,3.200000e-01,9.167366e+05,0.022444,9.222814e+05,1.507983e+18
max,8.950000e+02,9.269800e+04,8.678978e+18,3.327150e+18,9.206705e+18,1.547012e+18,3.111738e+04,4.050610e+06,5.566263,4.050610e+06,1.508963e+18


In [ ]:
df_e